# Matrix multiplication tensor decompositions satisfying various symmetries
$\newcommand{\cyc}{{\tiny \bigtriangleup}}
\newcommand{\tp}{\intercal}
\newcommand{\ang}[1]{\langle #1 \rangle}
\newcommand{\id}{\textsf{id}}
\newcommand{\mmt}[1]{\mathcal{T}^{\ang{#1}}}$
$\mmt{n,m,k}:=\left[\begin{cases}1 & b=c \textrm{ and } d=e \textrm{ and} f=a \\ 0\end{cases}\right]_{a,b,c,d,e,f}$

Decomposition of rank $R$: $\mmt{n,m,k}_{a,b,c,d,e,f}=\sum_{r=0}^{R-1} A^{(r)}_{a,b}B^{(r)}_{c,d}C^{(r)}_{e,f}$

In [1]:
import z3, itertools as it, time, numpy as np

In [4]:
'''
decomp_form: a ndarray of z3 expressions
'''
def sol2arr(sol,N,R,decomp_form):
    assert decomp_form.shape==(3,R,N,N)
    out=[[[[None]*N for i in range(N)] for r in range(R)] for ax in range(3)]
    D={v.name():z3.is_true(sol.model()[v]) for v in sol.model()}
    sol_assignment_expr=z3.And([
        z3.Bool(name)==val
        for name,val in D.items()
    ])
    for ax in range(3):
        for r in range(R):
            for i in range(N):
                for j in range(N):
                    tmp=z3.Solver()
                    tmp.add(z3.And(sol_assignment_expr,decomp_form[ax][r][i][j]))
                    out[ax][r][i][j]=1 if tmp.check()==z3.sat else 0
    return out
def check_decomp_mod2(sol,N,R,decomp_form):
    arr=np.array(sol2arr(sol,N,R,decomp_form))
    out=np.zeros((N,)*6,dtype=int)
    for a,b,c in zip(*arr):
        out+=np.multiply.outer(a,np.multiply.outer(b,c))
    out%=2
    ans=np.zeros((N,)*6,dtype=int)
    for i,j,k in it.product(range(N),repeat=3):
        ans[(i,j,j,k,k,i)]=1
    assert np.array_equal(out,ans), (arr,out)

In [5]:
'''
flatten an ndarray of z3 expressions
'''
def expr_list(expr_arr):
    return np.array(expr_arr).flatten().tolist()

In [6]:
def Xor_(bools):
    out=False
    for a in bools:
        out=z3.Xor(out,a)
    return out

In [7]:
def mat_bools(N,mname):
    return np.array([[z3.Bool(f'{mname}{i},{j}') for j in range(N)] for i in range(N)])
def default_form(N,R):
    return np.array([
        [
            [[z3.Bool(f'{ax_name}{r},{i},{j}') for j in range(N)] for i in range(N)]
            for r in range(R)
        ]
        for ax,ax_name in enumerate('abc')
    ])

## pure tensor decomp SAT, no optimizations

In [8]:
def decomps_mmt(N,R,decomp_form):
    assert decomp_form.shape==(3,R,N,N)
    return z3.And([
        Xor_([
            z3.And(
                decomp_form[0][r][a][b],
                decomp_form[1][r][c][d],
                decomp_form[2][r][e][f],
            )
            for r in range(R)
        ])==(b==c and d==e and f==a)
        for a,b,c,d,e,f in it.product(range(N),repeat=6)
    ])

In [ ]:
# test pure tensor decomp
for N,R in list(it.product(range(1,3),range(1,8)))+list(it.product([3],range(1,22))):
    print(f'{N=} {R=}',end='')
    decomp_form=default_form(N,R)
    sol=z3.Solver()
    st=time.time()
    sol.add(decomps_mmt(N,R,decomp_form))
    ret=sol.check()
    print(f'\tt={time.time()-st}',(ret if ret==z3.unsat else str(ret).upper()))
    if ret==z3.sat:
        display(sol2arr(sol,N,R,decomp_form))
        check_decomp_mod2(sol,N,R,decomp_form)

## disallow all-0 parts

In [9]:
def decomps_mmt_nonzero(N,R,decomp_form):
    return z3.And(
        [z3.Or(expr_list(decomp_form[ax][r])) for ax in range(3) for r in range(R)]
        +[decomps_mmt(N,R,decomp_form)]
    )

In [ ]:
# test forced-nonzero tensor decomp
for N,R in list(it.product(range(1,3),range(1,8)))+list(it.product([3],range(1,22))):
    print(f'{N=} {R=}',end='')
    decomp_form=default_form(N,R)
    sol=z3.Solver()
    st=time.time()
    sol.add(decomps_mmt_nonzero(N,R,decomp_form))
    ret=sol.check()
    print(f'\tt={time.time()-st}',(ret if ret==z3.unsat else str(ret).upper()))
    if ret==z3.sat:
        display(sol2arr(sol,N,R,decomp_form))
        check_decomp_mod2(sol,N,R,decomp_form)

## strict lex ordering

In [10]:
def lex_lt_helper(A,B): # A,B python lists of z3 expressions
    for L in [A,B]:
        assert isinstance(L,list) and all(isinstance(b,z3.BoolRef) for b in L)
    assert len(A)==len(B), (len(A),len(B))
    N=len(A)
    if N==0:
        return False
    return z3.Or(
        z3.And(z3.Not(A[0]),B[0]),
        z3.And(A[0]==B[0],lex_lt_helper(A[1:],B[1:]))
    )

def lex_lt(A,B): # A,B nD-arrays of z3 expressions
    A,B=np.array(A),np.array(B)
    assert A.shape==B.shape
    A,B=expr_list(A),expr_list(B)
    for L in [A,B]:
        assert isinstance(L,list) and all(isinstance(b,z3.BoolRef) for b in L)
    return lex_lt_helper(A,B)

def bool_list(name,N):
    return [z3.Bool(f'{name}{i}') for i in range(N)]

for N in range(1,6):
    expr=lex_lt(bool_list('a',N),bool_list('b',N))
    st=time.time()
    for A in it.product([False,True],repeat=N):
        for B in it.product([False,True],repeat=N):
            eqA=z3.And([z3.Bool(f'a{i}')==a for i,a in enumerate(A)])
            eqB=z3.And([z3.Bool(f'b{i}')==b for i,b in enumerate(B)])
            s=z3.Solver()
            s.add(z3.And([expr,eqA,eqB]))
            assert (z3.sat if A<B else z3.unsat)==s.check(), (A,B)
    print(time.time()-st)

del bool_list

0.08268904685974121
0.26186323165893555
1.0632779598236084
4.118479013442993
16.514681100845337


In [11]:
def lex_lt_adj(seqs):
    seqs=list(seqs)
    #z3.And(z3.And(),z3.And()) does not work
    return True if len(seqs)<=1 else z3.And([
        lex_lt(seqs[i],seqs[i+1])
        for i in range(len(seqs)-1)
    ])
def lex_lt_all(expr,others):
    return z3.And([lex_lt(expr,oe) for oe in others])

# Symmetry restrictions

In [12]:
'''
convert a list of matrix triplets [(A_i,B_i,C_i)]_i
to 3 lists of matrices [[A_i]_i, [B_i]_i, [C_i]_i]
'''
def trilist2decomp(tris):
    return np.array([list(M) for M in zip(*tris)])

## Cyclic symmetry
Orbits:
* $\Theta_\id(D):=[(D,D,D)]$
* $\Theta_\cyc((A,B,C)):=[(A,B,C),(B,C,A),(C,A,B)]$

IN GENERAL: we can always force the representative of an orbit we choose to be the lex. minimum in its orbit

(otherwise we could change our representative to be the lex. min element in its orbit, and still have the same orbit)

in fact, we can make this inequality strict, since if two elements of an orbit are equal, then the entire orbit is a (multiplied version of) a simpler type of orbit
$\newcommand{\lexle}{\le_{\textrm{lex}}}
\newcommand{\lexlt}{\lt_{\textrm{lex}}}$
* $(A,B,C)\lexlt(B,C,A) \ \& \ (A,B,C)\lexlt(C,A,B)$

In [15]:
def cycsymm_decomp(N,R1,R3):
    def o3rep(r3):
        assert r3 in range(R3)
        return (mat_bools(N,f'a{r3},'),mat_bools(N,f'b{r3},'),mat_bools(N,f'c{r3},'))
    def o3(r3):
        A,B,C=o3rep(r3)
        return [(A,B,C),(B,C,A),(C,A,B)]
    decomp_form=trilist2decomp([
        (mat_bools(N,f'd{r1},'),)*3
        for r1 in range(R1)
    ]+[
        tri
        for r3 in range(R3)
        for tri in o3(r3)
    ])
    return z3.And([
        # ordering
        lex_lt_adj([mat_bools(N,f'd{r1},') for r1 in range(R1)]),
        lex_lt_adj([o3rep(r3)[:2] for r3 in range(R3)]),
        
        # canonical orbit representatives
        z3.And([
            lex_lt_all(
                o3rep(r3),
                o3(r3)[1:]
            )
            for r3 in range(R3)
        ]),
        decomps_mmt_nonzero(N,R1+3*R3,decomp_form)
    ]),decomp_form

### cyclic: rank $\le 15$ impossible for $\langle 3,3,3\rangle$

In [16]:
for N,R in list(it.product(range(1,3),range(1,8)))+list(it.product([3],range(1,22))):
    print(f'{N=} {R=}')
    for R1,R3 in it.product(range(R+1),repeat=2):
        if R1+3*R3==R:
            print(f'\t{(R1,R3)=}',end=' ')
            expr,form=cycsymm_decomp(N,R1,R3)
            sol=z3.Solver()
            sol.add(expr)
            
            st=time.time()
            ret=sol.check()
            print(f't={time.time()-st}',(ret if ret==z3.unsat else str(ret).upper()))
            if ret==z3.sat:
                display(sol2arr(sol,N,R,form))
                check_decomp_mod2(sol,N,R,form)

N=1 R=1
	(R1,R3)=(1, 0) t=0.012539863586425781 SAT


[[[[1]]], [[[1]]], [[[1]]]]

N=1 R=2
	(R1,R3)=(2, 0) t=0.009334802627563477 unsat
N=1 R=3
	(R1,R3)=(0, 1) t=0.009007930755615234 unsat
	(R1,R3)=(3, 0) t=0.0077610015869140625 unsat
N=1 R=4
	(R1,R3)=(1, 1) t=0.008795976638793945 unsat
	(R1,R3)=(4, 0) t=0.008542060852050781 unsat
N=1 R=5
	(R1,R3)=(2, 1) t=0.0076978206634521484 unsat
	(R1,R3)=(5, 0) t=0.008008956909179688 unsat
N=1 R=6
	(R1,R3)=(0, 2) t=0.008226156234741211 unsat
	(R1,R3)=(3, 1) t=0.007635831832885742 unsat
	(R1,R3)=(6, 0) t=0.00758814811706543 unsat
N=1 R=7
	(R1,R3)=(1, 2) t=0.008457183837890625 unsat
	(R1,R3)=(4, 1) t=0.007893085479736328 unsat
	(R1,R3)=(7, 0) t=0.00804591178894043 unsat
N=2 R=1
	(R1,R3)=(1, 0) t=0.008419036865234375 unsat
N=2 R=2
	(R1,R3)=(2, 0) t=0.009213924407958984 unsat
N=2 R=3
	(R1,R3)=(0, 1) t=0.010700702667236328 unsat
	(R1,R3)=(3, 0) t=0.009328842163085938 unsat
N=2 R=4
	(R1,R3)=(1, 1) t=0.012927055358886719 unsat
	(R1,R3)=(4, 0) t=0.010828971862792969 unsat
N=2 R=5
	(R1,R3)=(2, 1) t=0.012470006942749023 unsat
	(R1,R3)=(5,

[[[[1, 0], [0, 1]],
  [[0, 0], [0, 1]],
  [[1, 0], [1, 0]],
  [[1, 1], [0, 0]],
  [[0, 0], [1, 1]],
  [[1, 0], [0, 0]],
  [[0, 1], [0, 1]]],
 [[[1, 0], [0, 1]],
  [[1, 0], [1, 0]],
  [[1, 1], [0, 0]],
  [[0, 0], [0, 1]],
  [[1, 0], [0, 0]],
  [[0, 1], [0, 1]],
  [[0, 0], [1, 1]]],
 [[[1, 0], [0, 1]],
  [[1, 1], [0, 0]],
  [[0, 0], [0, 1]],
  [[1, 0], [1, 0]],
  [[0, 1], [0, 1]],
  [[0, 0], [1, 1]],
  [[1, 0], [0, 0]]]]

	(R1,R3)=(4, 1) t=0.012614011764526367 SAT


[[[[0, 0], [1, 1]],
  [[0, 1], [0, 1]],
  [[0, 1], [1, 1]],
  [[1, 0], [0, 0]],
  [[0, 0], [1, 0]],
  [[1, 1], [1, 1]],
  [[0, 1], [0, 0]]],
 [[[0, 0], [1, 1]],
  [[0, 1], [0, 1]],
  [[0, 1], [1, 1]],
  [[1, 0], [0, 0]],
  [[1, 1], [1, 1]],
  [[0, 1], [0, 0]],
  [[0, 0], [1, 0]]],
 [[[0, 0], [1, 1]],
  [[0, 1], [0, 1]],
  [[0, 1], [1, 1]],
  [[1, 0], [0, 0]],
  [[0, 1], [0, 0]],
  [[0, 0], [1, 0]],
  [[1, 1], [1, 1]]]]

	(R1,R3)=(7, 0) t=0.010354042053222656 unsat
N=3 R=1
	(R1,R3)=(1, 0) t=0.00975799560546875 unsat
N=3 R=2
	(R1,R3)=(2, 0) t=0.014693975448608398 unsat
N=3 R=3
	(R1,R3)=(0, 1) t=0.022670984268188477 unsat
	(R1,R3)=(3, 0) t=0.016824007034301758 unsat
N=3 R=4
	(R1,R3)=(1, 1) t=0.02621912956237793 unsat
	(R1,R3)=(4, 0) t=0.016787290573120117 unsat
N=3 R=5
	(R1,R3)=(2, 1) t=0.03331112861633301 unsat
	(R1,R3)=(5, 0) t=0.02021193504333496 unsat
N=3 R=6
	(R1,R3)=(0, 2) t=0.08370614051818848 unsat
	(R1,R3)=(3, 1) t=0.03710794448852539 unsat
	(R1,R3)=(6, 0) t=0.02404618263244629 unsat
N=3 R=7
	(R1,R3)=(1, 2) t=0.13175702095031738 unsat
	(R1,R3)=(4, 1) t=0.03850197792053223 unsat
	(R1,R3)=(7, 0) t=0.023760318756103516 unsat
N=3 R=8
	(R1,R3)=(2, 2) t=0.2355649471282959 unsat
	(R1,R3)=(5, 1) t=0.041426897048950195 unsat
	(R1,R3)=(8, 0) t=0.028219223022460938 unsat
N=3 R=9
	(R1,R3)=(0, 3) t=0.3768758773803711 unsat
	(R1,R3)=(3, 2) t=0.6024148464202881 unsat
	(R1,R3)=(6, 1) t=0.045427799224853516 unsa


KeyboardInterrupt



## Cyclic + transpose symmetry
$\cyc((A,B,C)):=(B,C,A),\ \tp((A,B,C)):=(A^\intercal,C^\intercal,B^\intercal)$

$G:=\ang{\cyc,\tp}\cong S_3$

$\cyc\tp=\tp\cyc^{-1} \Rightarrow G=\ang{\cyc}\ang{\tp}=\ang{\tp}\ang{\cyc}$

**All possible $\Theta:=G(t:=(A,B,C))$**: enumerate via stabilizer
* $\id$: $\Theta=\{(A,B,C),(B,C,A),(C,A,B),
                    (A^\intercal,C^\intercal,B^\intercal),
                    (C^\intercal,B^\intercal,A^\intercal),
                    (B^\intercal,A^\intercal,C^\intercal)\}$
* $\ang{\cyc}$: $\cyc(t)=t \Rightarrow t=(D,D,D)
    \Rightarrow \Theta=\{(D,D,D),(D^\intercal,D^\intercal,D^\intercal)\}$
    
* $\ang{\cyc^k \tp}$ for some fixed $k$:
    $t=(\cyc^k\tp)(t)=(\tp\cyc^{-k})(t)
    \Rightarrow \tp(t)=\cyc^{-k}(t)
    \Rightarrow \Theta=\ang{\cyc}(t)$
    * $k=0$: $(A,B,C)=(A^\intercal,C^\intercal,B^\intercal)
    \Rightarrow t=(S=S^\intercal,H,H^\intercal)$
    * $k=1$: $(A,B,C)=(C^\intercal,B^\intercal,A^\intercal)
    \Rightarrow t=(H^\intercal,S=S^\intercal,H)$
    * $k=2$: $t=(H,H^\intercal,S=S^\intercal)$
    
    in general, $t=(\cyc^k\tp)(t)$ $\Rightarrow$ $t':=\cyc^k(t)$ satisfies $t'=\tp(t')$
    
    $\Rightarrow \Theta=\ang{\cyc}(t')=\{(S,H,H^\intercal),(H,H^\intercal,S),(H^\intercal,S,H)\}$ for $S=S^\intercal$
* $G$: $\Theta=\{(Z=Z^\intercal,Z,Z)\}$

order restrictions WLOG:
* $(D,D,D)\lexlt (D^\intercal,D^\intercal,D^\intercal)
    \equiv D\lexlt D^\intercal$
* difficult to make such a reduction for $([S,H,H^\intercal),(H,H^\intercal,S),(H^\intercal,S,H)]$ since the parameters for each representative are asymmetric
* $(A,B,C)\lexlt (B,C,A),\ (C,A,B),\ 
    (A^\intercal,C^\intercal,B^\intercal),\ 
    (C^\intercal,B^\intercal,A^\intercal),\ 
    (B^\intercal,A^\intercal,C^\intercal)$
    
even better:
* force $(A_i,B_i)$ to be strictly lex incr: $(A,B,C)+(A,B,C') \rightarrow (A,B,C+C')$
* force $H_i$ to be strictly lex incr: $(S=S^\intercal,H,H^\intercal)+(S'=S'^\intercal,H,H^\intercal)
    \rightarrow (S+S'=(S+S')^\intercal,H,H^\intercal)$

In [20]:
symm_mat_bools=lambda N,name:np.array([[z3.Bool(f'{name}{min(r,c)},{max(r,c)}') for c in range(N)] for r in range(N)])
def lrot(A,s):
    s%=len(A)
    return A[s:]+A[:s]
def tri_tp(tri,n_times):
    n_times%=2
    a,b,c=tri
    return (c.T,b.T,a.T) if n_times==1 else tri
def cyctpsymm_decomp(N,R1,R2,R3,R6):
    def o6rep(r6):
        return (mat_bools(N,f'a{r6},'),mat_bools(N,f'b{r6},'),mat_bools(N,f'c{r6},'))
    def o6(r6):
        return [
            lrot( tri_tp( o6rep(r6) , tp ) , s )
            for tp in range(2)
            for s in range(3)
        ]
    decomp_form=trilist2decomp([
        (symm_mat_bools(N,f'z{r1},'),)*3
        for r1 in range(R1)
    ]+[
        tri
        for r2 in range(R2)
        for tri in [
            (mat_bools(N,f'd{r2},'),)*3,
            (mat_bools(N,f'd{r2},').T,)*3
        ]
    ]+[
        lrot( (symm_mat_bools(N,f's{r3},'),mat_bools(N,f'h{r3},'),mat_bools(N,f'h{r3},').T) , s )
        for r3 in range(R3)
        for s in range(3)
    ]+[
        tri
        for r6 in range(R6)
        for tri in o6(r6)
    ])
    return z3.And([
        # ordering
        lex_lt_adj([symm_mat_bools(N,f'z{r1},') for r1 in range(R1)]),
        lex_lt_adj([mat_bools(N,f'd{r2},') for r2 in range(R2)]),
        # (S,H,H.T),(S',H,H.T) --> (S+S',H,H.T); S,S' symmetric --> S+S' symmetric
        lex_lt_adj([mat_bools(N,f'h{r3},') for r3 in range(R3)]),
        # (A,B,C),(A,B,C') --> (A,B,C+C')
        lex_lt_adj([[mat_bools(N,f'a{r6},'),mat_bools(N,f'b{r6},')] for r6 in range(R6)]),
        
        # orbit reprsentatives
        # (D,D,D) minimum among <tp>*((D,D,D))
        z3.And([
            lex_lt(mat_bools(N,f'd{r2},'),mat_bools(N,f'd{r2},').T)
            for r2 in range(R2)
        ]),
        # (A,B,C) min among <cyc,tp>*((A,B,C))
        z3.And([
            lex_lt_all(o6rep(r6),o6(r6)[1:])
            for r6 in range(R6)
        ]),
        decomps_mmt_nonzero(N,R1+2*R2+3*R3+6*R6,decomp_form)
    ]),decomp_form

### cyclic + transpose: rank $\le 21$ impossible for $\langle 3,3,3\rangle$

In [22]:
for N,R in list(it.product(range(1,3),range(1,8)))+list(it.product([3],range(1,22))):
    print(f'{N=} {R=}')
    for R1,R2,R3,R6 in it.product(range(R+1),repeat=4):
        if R1+2*R2+3*R3+6*R6==R:
            print(f'\t{(R1,R2,R3,R6)=}',end=' ')
            expr,form=cyctpsymm_decomp(N,R1,R2,R3,R6)
            sol=z3.Solver()
            sol.add(expr)
            st=time.time()
            ret=sol.check()
            print(f't={time.time()-st}',(ret if ret==z3.unsat else str(ret).upper()))
            if ret==z3.sat:
                display(sol2arr(sol,N,R,form))
                check_decomp_mod2(sol,N,R,form)

N=1 R=1
	(R1,R2,R3,R6)=(1, 0, 0, 0) t=0.013056039810180664 SAT


[[[[1]]], [[[1]]], [[[1]]]]

N=1 R=2
	(R1,R2,R3,R6)=(0, 1, 0, 0) t=0.011779069900512695 unsat
	(R1,R2,R3,R6)=(2, 0, 0, 0) t=0.010685205459594727 unsat
N=1 R=3
	(R1,R2,R3,R6)=(0, 0, 1, 0) t=0.01055002212524414 SAT


[[[[1]], [[1]], [[1]]], [[[1]], [[1]], [[1]]], [[[1]], [[1]], [[1]]]]

	(R1,R2,R3,R6)=(1, 1, 0, 0) t=0.007423877716064453 unsat
	(R1,R2,R3,R6)=(3, 0, 0, 0) t=0.007467985153198242 unsat
N=1 R=4
	(R1,R2,R3,R6)=(0, 2, 0, 0) t=0.00873708724975586 unsat
	(R1,R2,R3,R6)=(1, 0, 1, 0) t=0.008232831954956055 unsat
	(R1,R2,R3,R6)=(2, 1, 0, 0) t=0.0074350833892822266 unsat
	(R1,R2,R3,R6)=(4, 0, 0, 0) t=0.00744318962097168 unsat
N=1 R=5
	(R1,R2,R3,R6)=(0, 1, 1, 0) t=0.007421970367431641 unsat
	(R1,R2,R3,R6)=(1, 2, 0, 0) t=0.007435798645019531 unsat
	(R1,R2,R3,R6)=(2, 0, 1, 0) t=0.007528066635131836 unsat
	(R1,R2,R3,R6)=(3, 1, 0, 0) t=0.00798797607421875 unsat
	(R1,R2,R3,R6)=(5, 0, 0, 0) t=0.007849931716918945 unsat
N=1 R=6
	(R1,R2,R3,R6)=(0, 0, 0, 1) t=0.009456157684326172 unsat
	(R1,R2,R3,R6)=(0, 0, 2, 0) t=0.009760141372680664 unsat
	(R1,R2,R3,R6)=(0, 3, 0, 0) t=0.009849071502685547 unsat
	(R1,R2,R3,R6)=(1, 1, 1, 0) t=0.010661125183105469 unsat
	(R1,R2,R3,R6)=(2, 2, 0, 0) t=0.00914907455444336 unsat
	(R1,R2,R3,R6)=(3, 0, 1, 0) t=0.007650852203369141 unsat
	(R1,R2,R3

...

	(R1,R2,R3,R6)=(3, 2, 0, 0) t=0.009639739990234375 unsat
	(R1,R2,R3,R6)=(4, 0, 1, 0) t=0.011312007904052734 unsat
	(R1,R2,R3,R6)=(5, 1, 0, 0) t=0.010277748107910156 unsat
	(R1,R2,R3,R6)=(7, 0, 0, 0) t=0.008101224899291992 unsat
N=3 R=1
	(R1,R2,R3,R6)=(1, 0, 0, 0) t=0.009337186813354492 unsat
N=3 R=2
	(R1,R2,R3,R6)=(0, 1, 0, 0) t=0.008223295211791992 unsat
	(R1,R2,R3,R6)=(2, 0, 0, 0) t=0.008560895919799805 unsat
N=3 R=3
	(R1,R2,R3,R6)=(0, 0, 1, 0) t=0.01834392547607422 unsat
	(R1,R2,R3,R6)=(1, 1, 0, 0) t=0.01780986785888672 unsat
	(R1,R2,R3,R6)=(3, 0, 0, 0) t=0.011481285095214844 unsat
N=3 R=4
	(R1,R2,R3,R6)=(0, 2, 0, 0) t=0.00900411605834961 unsat
	(R1,R2,R3,R6)=(1, 0, 1, 0) t=0.019696950912475586 unsat
	(R1,R2,R3,R6)=(2, 1, 0, 0) t=0.013190984725952148 unsat
	(R1,R2,R3,R6)=(4, 0, 0, 0) t=0.008939981460571289 unsat
N=3 R=5
	(R1,R2,R3,R6)=(0, 1, 1, 0) t=0.02181386947631836 unsat
	(R1,R2,R3,R6)=(1, 2, 0, 0) t=0.016744136810302734 unsat
	(R1,R2,R3,R6)=(2, 0, 1, 0) t=0.02209782600402832 un

	(R1,R2,R3,R6)=(3, 5, 0, 0) t=0.06541585922241211 unsat
	(R1,R2,R3,R6)=(4, 0, 1, 1) t=0.13566899299621582 unsat
	(R1,R2,R3,R6)=(4, 0, 3, 0) t=0.2373661994934082 unsat
	(R1,R2,R3,R6)=(4, 3, 1, 0) t=0.046195030212402344 unsat
	(R1,R2,R3,R6)=(5, 1, 0, 1) t=0.05286717414855957 unsat
	(R1,R2,R3,R6)=(5, 1, 2, 0) t=0.12791681289672852 unsat
	(R1,R2,R3,R6)=(5, 4, 0, 0) t=0.032157182693481445 unsat
	(R1,R2,R3,R6)=(6, 2, 1, 0) t=0.042279958724975586 unsat
	(R1,R2,R3,R6)=(7, 0, 0, 1) t=0.05372905731201172 unsat
	(R1,R2,R3,R6)=(7, 0, 2, 0) t=0.05085301399230957 unsat
	(R1,R2,R3,R6)=(7, 3, 0, 0) t=0.028912782669067383 unsat
	(R1,R2,R3,R6)=(8, 1, 1, 0) t=0.03568911552429199 unsat
	(R1,R2,R3,R6)=(9, 2, 0, 0) t=0.027806997299194336 unsat
	(R1,R2,R3,R6)=(10, 0, 1, 0) t=0.03441596031188965 unsat
	(R1,R2,R3,R6)=(11, 1, 0, 0) t=0.024589061737060547 unsat
	(R1,R2,R3,R6)=(13, 0, 0, 0) t=0.013796806335449219 unsat
N=3 R=14
	(R1,R2,R3,R6)=(0, 1, 0, 2) t=0.07140588760375977 unsat
	(R1,R2,R3,R6)=(0, 1, 2, 1) t=

	(R1,R2,R3,R6)=(0, 4, 3, 0) t=0.18244314193725586 unsat
	(R1,R2,R3,R6)=(0, 7, 1, 0) t=0.054924964904785156 unsat
	(R1,R2,R3,R6)=(1, 2, 0, 2) t=0.1804969310760498 unsat
	(R1,R2,R3,R6)=(1, 2, 2, 1) t=0.7562010288238525 unsat
	(R1,R2,R3,R6)=(1, 2, 4, 0) t=26.90453290939331 unsat
	(R1,R2,R3,R6)=(1, 5, 0, 1) t=0.07286286354064941 unsat
	(R1,R2,R3,R6)=(1, 5, 2, 0) t=0.12816596031188965 unsat
	(R1,R2,R3,R6)=(1, 8, 0, 0) t=0.041268110275268555 unsat
	(R1,R2,R3,R6)=(2, 0, 1, 2) t=0.22034502029418945 unsat
	(R1,R2,R3,R6)=(2, 0, 3, 1) t=75.50810408592224 unsat
	(R1,R2,R3,R6)=(2, 0, 5, 0) t=94.04775404930115 unsat
	(R1,R2,R3,R6)=(2, 3, 1, 1) t=0.19559431076049805 unsat
	(R1,R2,R3,R6)=(2, 3, 3, 0) t=2.711452007293701 unsat
	(R1,R2,R3,R6)=(2, 6, 1, 0) t=0.06621384620666504 unsat
	(R1,R2,R3,R6)=(3, 1, 0, 2) t=0.08641719818115234 unsat
	(R1,R2,R3,R6)=(3, 1, 2, 1) t=15.758748769760132 unsat
	(R1,R2,R3,R6)=(3, 1, 4, 0) t=45.930707931518555 unsat
	(R1,R2,R3,R6)=(3, 4, 0, 1) t=0.06915283203125 unsat
	(R1,

	(R1,R2,R3,R6)=(4, 6, 1, 0) t=0.06431293487548828 unsat
	(R1,R2,R3,R6)=(5, 1, 0, 2) t=0.09112191200256348 unsat
	(R1,R2,R3,R6)=(5, 1, 2, 1) t=19.1216938495636 unsat
	(R1,R2,R3,R6)=(5, 1, 4, 0) t=49.421345949172974 unsat
	(R1,R2,R3,R6)=(5, 4, 0, 1) t=0.12494206428527832 unsat
	(R1,R2,R3,R6)=(5, 4, 2, 0) t=0.17564010620117188 unsat
	(R1,R2,R3,R6)=(5, 7, 0, 0) t=0.0720210075378418 unsat
	(R1,R2,R3,R6)=(6, 2, 1, 1) t=0.08590912818908691 unsat
	(R1,R2,R3,R6)=(6, 2, 3, 0) t=3.4499497413635254 unsat
	(R1,R2,R3,R6)=(6, 5, 1, 0) t=0.07032513618469238 unsat
	(R1,R2,R3,R6)=(7, 0, 0, 2) t=0.11134910583496094 unsat
	(R1,R2,R3,R6)=(7, 0, 2, 1) t=2.708249092102051 unsat
	(R1,R2,R3,R6)=(7, 0, 4, 0) t=4.519016981124878 unsat
	(R1,R2,R3,R6)=(7, 3, 0, 1) t=0.09113025665283203 unsat
	(R1,R2,R3,R6)=(7, 3, 2, 0) t=0.1518251895904541 unsat
	(R1,R2,R3,R6)=(7, 6, 0, 0) t=0.04099774360656738 unsat
	(R1,R2,R3,R6)=(8, 1, 1, 1) t=0.14559507369995117 unsat
	(R1,R2,R3,R6)=(8, 1, 3, 0) t=1.2809460163116455 unsat
	(R1

	(R1,R2,R3,R6)=(3, 3, 0, 2) t=0.10054993629455566 unsat
	(R1,R2,R3,R6)=(3, 3, 2, 1) t=35.50213408470154 unsat
	(R1,R2,R3,R6)=(3, 3, 4, 0) t=565.9668219089508 unsat
	(R1,R2,R3,R6)=(3, 6, 0, 1) t=0.08414077758789062 unsat
	(R1,R2,R3,R6)=(3, 6, 2, 0) t=0.1766648292541504 unsat
	(R1,R2,R3,R6)=(3, 9, 0, 0) t=0.05147218704223633 unsat
	(R1,R2,R3,R6)=(4, 1, 1, 2) t=0.22397112846374512 unsat
	(R1,R2,R3,R6)=(4, 1, 3, 1) t=5803.420874118805 unsat
	(R1,R2,R3,R6)=(4, 1, 5, 0) t=129.0978491306305 unsat
	(R1,R2,R3,R6)=(4, 4, 1, 1) t=0.10873293876647949 unsat
	(R1,R2,R3,R6)=(4, 4, 3, 0) t=5.486916780471802 unsat
	(R1,R2,R3,R6)=(4, 7, 1, 0) t=0.09368515014648438 unsat
	(R1,R2,R3,R6)=(5, 2, 0, 2) t=0.11777114868164062 unsat
	(R1,R2,R3,R6)=(5, 2, 2, 1) t=56.69844102859497 unsat
	(R1,R2,R3,R6)=(5, 2, 4, 0) t=660.37056016922 unsat
	(R1,R2,R3,R6)=(5, 5, 0, 1) t=0.08162999153137207 unsat
	(R1,R2,R3,R6)=(5, 5, 2, 0) t=0.16507697105407715 unsat
	(R1,R2,R3,R6)=(5, 8, 0, 0) t=0.04914093017578125 unsat
	(R1,R2,R

## Cyclic + involute-sandwich symmetry

$\ang{\cyc,\phi_F}$, $F^2=I$

$\phi_F((A,B,C)):=(FAF^{-1},FBF^{-1},FCF^{-1})$

$\phi_F ^2 = \id$, $\phi_F\ne\id$
$\Rightarrow G:=\ang{\cyc,\phi_F}\cong C_2\times C_3 \cong C_6$

orbit of matrix triplet, given stabilizer $\dots$:
* $\id$: $[(A,B,C),(B,C,A),(C,A,B), (FAF^{-1},FBF^{-1},FCF^{-1}),(FBF^{-1},FCF^{-1},FAF^{-1}),(FCF^{-1},FAF^{-1},FBF^{-1})]$
* $\cyc$: $[(D,D,D),(FDF^{-1},FDF^{-1},FDF^{-1})]$
* $\phi_F$: $[(X,Y,Z),(Y,Z,X),(Z,X,Y)]$ s.t. $(X,Y,Z)=(FXF^{-1},FYF^{-1},FZF^{-1})$
* $G$: $[(U,U,U)]$ s.t. $U=FUF^{-1}$

In [23]:
def cycswsymm_decomp(N,F,R1,R2,R3,R6):
    assert F.shape==(N,N) and F.dtype==np.int64 and ((F>=0)&(F<2)).all(), F
    assert np.array_equal((F@F)%2,np.eye(N,dtype=int)), F
    # cast to int to get bool, not numpy.bool_
    F=[[int(F[r][c])==1 for c in range(N)] for r in range(N)]
    def swmat(M): # return F@M@F
        assert M.shape==(N,N)
        return np.array([
            [
                Xor_([z3.And(F[i][a],M[a][b],F[b][j]) for a in range(N) for b in range(N)])
                for j in range(N)
            ]
            for i in range(N)
        ])
    def tri_sw(tri,n_times):
        n_times%=2
        a,b,c=tri
        return (swmat(a),swmat(b),swmat(c)) if n_times==1 else tri
    def o3rep(r3):
        assert r3 in range(R3)
        return (mat_bools(N,f'x{r3},'),mat_bools(N,f'y{r3},'),mat_bools(N,f'z{r3},'))
    def o3(r3):
        rep=o3rep(r3)
        return [lrot(rep,cyc) for cyc in range(3)]
    def o6rep(r6):
        assert r6 in range(R6)
        return (mat_bools(N,f'a{r6},'),mat_bools(N,f'b{r6},'),mat_bools(N,f'c{r6},'))
    def o6(r6):
        rep=o6rep(r6)
        return [lrot(tri_sw(rep,sw),cyc) for sw in range(2) for cyc in range(3)]
    decomp_form=trilist2decomp([
        (mat_bools(N,f'u{r1},'),)*3
        for r1 in range(R1)
    ]+[
        tri
        for r2 in range(R2)
        for tri in [
            (mat_bools(N,f'd{r2},'),)*3,
            (swmat(mat_bools(N,f'd{r2},')),)*3
        ]
    ]+[
        tri
        for r3 in range(R3)
        for tri in o3(r3)
    ]+[
        tri
        for r6 in range(R6)
        for tri in o6(r6)
    ])
    
    def eq_mat(A,B):
        assert A.shape==(N,N) and B.shape==(N,N), (A,B)
        return z3.And([A[i][j]==B[i][j] for i in range(N) for j in range(N)])
    return z3.And([
        # symmetry requirements
        z3.And([
            eq_mat(mat_bools(N,f'u{r1},'),swmat(mat_bools(N,f'u{r1},')))
            for r1 in range(R1)
        ]),
        z3.And([
            z3.And(
                eq_mat(mat_bools(N,f'x{r3},'),swmat(mat_bools(N,f'x{r3},'))),
                eq_mat(mat_bools(N,f'y{r3},'),swmat(mat_bools(N,f'y{r3},'))),
                eq_mat(mat_bools(N,f'z{r3},'),swmat(mat_bools(N,f'z{r3},')))
            )
            for r3 in range(R3)
        ]),
        
        # ordering
        lex_lt_adj([mat_bools(N,f'u{r1},') for r1 in range(R1)]),
        lex_lt_adj([mat_bools(N,f'd{r2},') for r2 in range(R2)]),
        lex_lt_adj([o3rep(r3)[:2] for r3 in range(R3)]),
        lex_lt_adj([o6rep(r6)[:2] for r6 in range(R6)]),
        
        # canonical orbit representatives
        z3.And([
            lex_lt(
                mat_bools(N,f'd{r2},'),
                swmat(mat_bools(N,f'd{r2},'))
            )
            for r2 in range(R2)
        ]),
        z3.And([
            lex_lt_all(
                o3rep(r3),
                o3(r3)[1:]
            )
            for r3 in range(R3)
        ]),
        z3.And([
            lex_lt_all(
                o6rep(r6),
                o6(r6)[1:]
            )
            for r6 in range(R6)
        ]),
        decomps_mmt_nonzero(N,R1+2*R2+3*R3+6*R6,decomp_form)
    ]),decomp_form

### cyclic + $\begin{bmatrix}1&1&0\\0&1&0\\0&0&1\end{bmatrix}$-sandwich: rank $\le 21$ impossible for $\langle 3,3,3\rangle$

In [24]:
for R,N,F in [
    (r,1,np.eye(1,dtype=int))
    for r in range(1,8)
]+[
    (r,2,np.array([[0,1],[1,0]]))
    for r in range(1,8)
]+[
    (r,3,np.array([[1,1,0],[0,1,0],[0,0,1]]))
    for r in range(1,22)
]:
    print(f'{N=} {R=} {F=}')
    for R1,R2,R3,R6 in it.product(range(R+1),repeat=4):
        if R1+2*R2+3*R3+6*R6==R:
            print(f'\t{(R1,R2,R3,R6)=}',end='')
            expr,form=cycswsymm_decomp(N,F,R1,R2,R3,R6)
            sol=z3.Solver()
            sol.add(expr)
            st=time.time()
            ret=sol.check()
            print(f't={time.time()-st}',(ret if ret==z3.unsat else str(ret).upper()))
            if ret==z3.sat:
                display(sol2arr(sol,N,R,form))
                check_decomp_mod2(sol,N,R,form)

N=1 R=1 F=array([[1]])
	(R1,R2,R3,R6)=(1, 0, 0, 0)t=0.01239013671875 SAT


[[[[1]]], [[[1]]], [[[1]]]]

N=1 R=2 F=array([[1]])
	(R1,R2,R3,R6)=(0, 1, 0, 0)t=0.009009838104248047 unsat
	(R1,R2,R3,R6)=(2, 0, 0, 0)t=0.009070158004760742 unsat
N=1 R=3 F=array([[1]])
	(R1,R2,R3,R6)=(0, 0, 1, 0)t=0.009142875671386719 unsat
	(R1,R2,R3,R6)=(1, 1, 0, 0)t=0.009633064270019531 unsat
	(R1,R2,R3,R6)=(3, 0, 0, 0)t=0.008415937423706055 unsat
N=1 R=4 F=array([[1]])
	(R1,R2,R3,R6)=(0, 2, 0, 0)t=0.009078025817871094 unsat
	(R1,R2,R3,R6)=(1, 0, 1, 0)t=0.008786916732788086 unsat
	(R1,R2,R3,R6)=(2, 1, 0, 0)t=0.008337020874023438 unsat
	(R1,R2,R3,R6)=(4, 0, 0, 0)t=0.008085966110229492 unsat
N=1 R=5 F=array([[1]])
	(R1,R2,R3,R6)=(0, 1, 1, 0)t=0.008307218551635742 unsat
	(R1,R2,R3,R6)=(1, 2, 0, 0)t=0.008468151092529297 unsat
	(R1,R2,R3,R6)=(2, 0, 1, 0)t=0.008482694625854492 unsat
	(R1,R2,R3,R6)=(3, 1, 0, 0)t=0.008093833923339844 unsat
	(R1,R2,R3,R6)=(5, 0, 0, 0)t=0.007686138153076172 unsat
N=1 R=6 F=array([[1]])
	(R1,R2,R3,R6)=(0, 0, 0, 1)t=0.00886082649230957 unsat
	(R1,R2,R3,R6)=(0, 0, 2, 0)t=0.011140108108520

[[[[1, 0], [0, 1]],
  [[0, 0], [0, 1]],
  [[1, 0], [1, 0]],
  [[1, 1], [0, 0]],
  [[1, 0], [0, 0]],
  [[0, 1], [0, 1]],
  [[0, 0], [1, 1]]],
 [[[1, 0], [0, 1]],
  [[1, 0], [1, 0]],
  [[1, 1], [0, 0]],
  [[0, 0], [0, 1]],
  [[0, 1], [0, 1]],
  [[0, 0], [1, 1]],
  [[1, 0], [0, 0]]],
 [[[1, 0], [0, 1]],
  [[1, 1], [0, 0]],
  [[0, 0], [0, 1]],
  [[1, 0], [1, 0]],
  [[0, 0], [1, 1]],
  [[1, 0], [0, 0]],
  [[0, 1], [0, 1]]]]

	(R1,R2,R3,R6)=(1, 0, 2, 0)t=0.01453399658203125 unsat
	(R1,R2,R3,R6)=(1, 3, 0, 0)t=0.011752128601074219 unsat
	(R1,R2,R3,R6)=(2, 1, 1, 0)t=0.016328811645507812 unsat
	(R1,R2,R3,R6)=(3, 2, 0, 0)t=0.010612010955810547 unsat
	(R1,R2,R3,R6)=(4, 0, 1, 0)t=0.012663125991821289 unsat
	(R1,R2,R3,R6)=(5, 1, 0, 0)t=0.01182699203491211 unsat
	(R1,R2,R3,R6)=(7, 0, 0, 0)t=0.011255025863647461 unsat
N=3 R=1 F=array([[1, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])
	(R1,R2,R3,R6)=(1, 0, 0, 0)t=0.010904073715209961 unsat
N=3 R=2 F=array([[1, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])
	(R1,R2,R3,R6)=(0, 1, 0, 0)t=0.009858846664428711 unsat
	(R1,R2,R3,R6)=(2, 0, 0, 0)t=0.013261079788208008 unsat
N=3 R=3 F=array([[1, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])
	(R1,R2,R3,R6)=(0, 0, 1, 0)t=0.024663209915161133 unsat
	(R1,R2,R3,R6)=(1, 1, 0, 0)t=0.01886892318725586 unsat
	(R1,R2,R3,R6)=(3, 0, 0, 0)t=0.017431020736694336 unsat
N=3 R=4 F=array([[1, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])
	(R1,R2,R3,R6)=

	(R1,R2,R3,R6)=(0, 2, 3, 0)t=0.09815716743469238 unsat
	(R1,R2,R3,R6)=(0, 5, 1, 0)t=0.07503795623779297 unsat
	(R1,R2,R3,R6)=(1, 0, 0, 2)t=1.101870059967041 unsat
	(R1,R2,R3,R6)=(1, 0, 2, 1)t=0.27104902267456055 unsat
	(R1,R2,R3,R6)=(1, 0, 4, 0)t=0.12034487724304199 unsat
	(R1,R2,R3,R6)=(1, 3, 0, 1)t=0.24319720268249512 unsat
	(R1,R2,R3,R6)=(1, 3, 2, 0)t=0.09381699562072754 unsat
	(R1,R2,R3,R6)=(1, 6, 0, 0)t=0.0548710823059082 unsat
	(R1,R2,R3,R6)=(2, 1, 1, 1)t=0.19456005096435547 unsat
	(R1,R2,R3,R6)=(2, 1, 3, 0)t=0.09964418411254883 unsat
	(R1,R2,R3,R6)=(2, 4, 1, 0)t=0.07599496841430664 unsat
	(R1,R2,R3,R6)=(3, 2, 0, 1)t=0.19849085807800293 unsat
	(R1,R2,R3,R6)=(3, 2, 2, 0)t=0.08948016166687012 unsat
	(R1,R2,R3,R6)=(3, 5, 0, 0)t=0.05191922187805176 unsat
	(R1,R2,R3,R6)=(4, 0, 1, 1)t=0.25328803062438965 unsat
	(R1,R2,R3,R6)=(4, 0, 3, 0)t=0.10124993324279785 unsat
	(R1,R2,R3,R6)=(4, 3, 1, 0)t=0.07780194282531738 unsat
	(R1,R2,R3,R6)=(5, 1, 0, 1)t=0.2756199836730957 unsat
	(R1,R2,R3,R6)

	(R1,R2,R3,R6)=(9, 2, 1, 0)t=0.08489203453063965 unsat
	(R1,R2,R3,R6)=(10, 0, 0, 1)t=0.2344367504119873 unsat
	(R1,R2,R3,R6)=(10, 0, 2, 0)t=0.09359383583068848 unsat
	(R1,R2,R3,R6)=(10, 3, 0, 0)t=0.05083513259887695 unsat
	(R1,R2,R3,R6)=(11, 1, 1, 0)t=0.0827798843383789 unsat
	(R1,R2,R3,R6)=(12, 2, 0, 0)t=0.050180673599243164 unsat
	(R1,R2,R3,R6)=(13, 0, 1, 0)t=0.06917405128479004 unsat
	(R1,R2,R3,R6)=(14, 1, 0, 0)t=0.05698418617248535 unsat
	(R1,R2,R3,R6)=(16, 0, 0, 0)t=0.051023006439208984 unsat
N=3 R=17 F=array([[1, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])
	(R1,R2,R3,R6)=(0, 1, 1, 2)t=5.292748928070068 unsat
	(R1,R2,R3,R6)=(0, 1, 3, 1)t=0.29764890670776367 unsat
	(R1,R2,R3,R6)=(0, 1, 5, 0)t=0.14397716522216797 unsat
	(R1,R2,R3,R6)=(0, 4, 1, 1)t=0.3676478862762451 unsat
	(R1,R2,R3,R6)=(0, 4, 3, 0)t=0.12275075912475586 unsat
	(R1,R2,R3,R6)=(0, 7, 1, 0)t=0.09876775741577148 unsat
	(R1,R2,R3,R6)=(1, 2, 0, 2)t=15.140315055847168 unsat
	(R1,R2,R3,R6)=(1, 2, 2, 1)t=0.3224067687988281 un

	(R1,R2,R3,R6)=(2, 4, 1, 1)t=0.32664918899536133 unsat
	(R1,R2,R3,R6)=(2, 4, 3, 0)t=0.16203904151916504 unsat
	(R1,R2,R3,R6)=(2, 7, 1, 0)t=0.11548399925231934 unsat
	(R1,R2,R3,R6)=(3, 2, 0, 2)t=26.39505100250244 unsat
	(R1,R2,R3,R6)=(3, 2, 2, 1)t=0.3360440731048584 unsat
	(R1,R2,R3,R6)=(3, 2, 4, 0)t=0.25267601013183594 unsat
	(R1,R2,R3,R6)=(3, 5, 0, 1)t=0.3389852046966553 unsat
	(R1,R2,R3,R6)=(3, 5, 2, 0)t=0.12877893447875977 unsat
	(R1,R2,R3,R6)=(3, 8, 0, 0)t=0.1086580753326416 unsat
	(R1,R2,R3,R6)=(4, 0, 1, 2)t=1.8154799938201904 unsat
	(R1,R2,R3,R6)=(4, 0, 3, 1)t=0.3757309913635254 unsat
	(R1,R2,R3,R6)=(4, 0, 5, 0)t=0.29346585273742676 unsat
	(R1,R2,R3,R6)=(4, 3, 1, 1)t=0.3381378650665283 unsat
	(R1,R2,R3,R6)=(4, 3, 3, 0)t=0.19486594200134277 unsat
	(R1,R2,R3,R6)=(4, 6, 1, 0)t=0.16333794593811035 unsat
	(R1,R2,R3,R6)=(5, 1, 0, 2)t=7.132327079772949 unsat
	(R1,R2,R3,R6)=(5, 1, 2, 1)t=0.35755300521850586 unsat
	(R1,R2,R3,R6)=(5, 1, 4, 0)t=0.1617751121520996 unsat
	(R1,R2,R3,R6)=(5, 4,

	(R1,R2,R3,R6)=(1, 4, 4, 0)t=0.13144707679748535 unsat
	(R1,R2,R3,R6)=(1, 7, 0, 1)t=0.27165794372558594 unsat
	(R1,R2,R3,R6)=(1, 7, 2, 0)t=0.12761187553405762 unsat
	(R1,R2,R3,R6)=(1, 10, 0, 0)t=0.08141493797302246 unsat
	(R1,R2,R3,R6)=(2, 2, 1, 2)t=22.38855004310608 unsat
	(R1,R2,R3,R6)=(2, 2, 3, 1)t=0.3199160099029541 unsat
	(R1,R2,R3,R6)=(2, 2, 5, 0)t=0.16501188278198242 unsat
	(R1,R2,R3,R6)=(2, 5, 1, 1)t=0.3204460144042969 unsat
	(R1,R2,R3,R6)=(2, 5, 3, 0)t=0.12054872512817383 unsat
	(R1,R2,R3,R6)=(2, 8, 1, 0)t=0.09686398506164551 unsat
	(R1,R2,R3,R6)=(3, 0, 0, 3)t=785.4879989624023 unsat
	(R1,R2,R3,R6)=(3, 0, 2, 2)t=2.6942858695983887 unsat
	(R1,R2,R3,R6)=(3, 0, 4, 1)t=0.334306001663208 unsat
	(R1,R2,R3,R6)=(3, 0, 6, 0)t=0.2856729030609131 unsat
	(R1,R2,R3,R6)=(3, 3, 0, 2)t=39.523285150527954 unsat
	(R1,R2,R3,R6)=(3, 3, 2, 1)t=0.2890799045562744 unsat
	(R1,R2,R3,R6)=(3, 3, 4, 0)t=0.12360429763793945 unsat
	(R1,R2,R3,R6)=(3, 6, 0, 1)t=0.2793290615081787 unsat
	(R1,R2,R3,R6)=(3, 6, 